# TAYSIR competition - Track 2 Example Extraction

### Welcome!

This is a notebook to show the structure of a code to participate to the competition.

You can also check the baseline notebook (available in the same archive) for more details about the TAYSIR models and how to use them.

## Prepare your environment

In [ ]:
#!pip install mlflow torch

In [6]:
!pip install --upgrade pymodelextractor

  Attempting uninstall: pymodelextractor
    Found existing installation: pymodelextractor 0.35.0
    Uninstalling pymodelextractor-0.35.0:
      Successfully uninstalled pymodelextractor-0.35.0


In [118]:
import torch
import mlflow
print('PyTorch version :', torch.__version__)
print('MLflow version :', mlflow.__version__)
import sys
print("Your python version:", sys.version)

PyTorch version : 2.1.0.dev20230403+cpu
MLflow version : 2.2.2
Your python version: 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:20:46) 
[GCC 9.4.0]


In [119]:
torch.set_num_threads(4)

This notebook was tested with:
* Torch version: 1.11.0+cu102
* MLFlow version: 1.25.1
* Python version: 3.8.10 [GCC 9.4.0]

Python versions starting at 3.7 are supposed to work (but have not been tested).
## Choosing the phase

First you must select one of the phases/datasets we provide

In [120]:
dataset_amount = 10
for ds in range(1,dataset_amount+1):
    DATASET = ds
    model_name = f"models/2.{DATASET}.taysir.model"
    model = mlflow.pytorch.load_model(model_name)
    
    print("\n")
    print("Model:", ds)
    print(model.eval())
    try:#RNN
        nb_letters = model.input_size -1
        cell_type = model.cell_type

        print("The alphabet contains", nb_letters, "symbols.")
        print("The type of the recurrent cells is", cell_type.__name__)
    except:
        nb_letters = model.distilbert.config.vocab_size
        print("The alphabet contains", nb_letters, "symbols.")
        print("The model is a transformer (DistilBertForSequenceClassification)")

2023/04/20 15:38:04 WARNING mlflow.pytorch: Stored model version '1.13.1+cpu' does not match installed PyTorch version '2.1.0.dev20230403+cpu'
2023/04/20 15:38:04 WARNING mlflow.pytorch: Stored model version '1.13.1+cpu' does not match installed PyTorch version '2.1.0.dev20230403+cpu'
2023/04/20 15:38:04 WARNING mlflow.pytorch: Stored model version '1.13.1+cpu' does not match installed PyTorch version '2.1.0.dev20230403+cpu'
2023/04/20 15:38:04 WARNING mlflow.pytorch: Stored model version '1.13.1+cpu' does not match installed PyTorch version '2.1.0.dev20230403+cpu'
2023/04/20 15:38:04 WARNING mlflow.pytorch: Stored model version '1.13.1+cpu' does not match installed PyTorch version '2.1.0.dev20230403+cpu'
2023/04/20 15:38:04 WARNING mlflow.pytorch: Stored model version '1.13.1+cpu' does not match installed PyTorch version '2.1.0.dev20230403+cpu'
2023/04/20 15:38:04 WARNING mlflow.pytorch: Stored model version '1.13.1+cpu' does not match installed PyTorch version '2.1.0.dev20230403+cpu'



Model: 1
TNetwork(
  23, 22, n_layers=2, neurons_per_layer=64, batch_size=64, patience=5, split_dense=True, task=lm
  (mach[0]): RNN(22, 64, batch_first=True)
  (mach[1]): RNN(64, 64, batch_first=True)
  (dense): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=22, bias=True)
    (2): Sigmoid()
    (3): Softmax(dim=-1)
  )
)
The alphabet contains 22 symbols.
The type of the recurrent cells is RNN


Model: 2
TNetwork(
  10, 9, n_layers=2, neurons_per_layer=256, cell_type=lstmx.LSTMx, batch_size=64, patience=5, split_dense=True, task=lm
  (mach[0]): LSTMx(
    9, 256, batch_first=True
    (drop_layer): Dropout(p=0, inplace=False)
    (forward_layers[0]): LSTMCell(9, 256)
  )
  (mach[1]): LSTMx(
    256, 256, batch_first=True
    (drop_layer): Dropout(p=0, inplace=False)
    (forward_layers[0]): LSTMCell(256, 256)
  )
  (dense): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): Linear(in

In [148]:
TRACK = 2 #always for this track
DATASET = 2

In [150]:
model_name = f"models/{TRACK}.{DATASET}.taysir.model"

model = mlflow.pytorch.load_model(model_name)
model.eval()

2023/04/21 15:43:30 WARNING mlflow.pytorch: Stored model version '1.13.1+cu117' does not match installed PyTorch version '2.1.0.dev20230403+cpu'


TNetwork(
  19, 2, patience=5, bidirectional=True, split_dense=True
  (mach[0]): RNN(18, 32, batch_first=True, bidirectional=True)
  (dense): Sequential(
    (0): Linear(in_features=64, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=2, bias=True)
    (2): Sigmoid()
    (3): Softmax(dim=-1)
  )
)

In [123]:
if not hasattr(model, 'distilbert'):#RNN
    nb_letters = model.input_size -1
    cell_type = model.cell_type

    print("The alphabet contains", nb_letters, "symbols.")
    print("The type of the recurrent cells is", cell_type.__name__)
else:
    nb_letters = model.distilbert.config.vocab_size
    print("The alphabet contains", nb_letters, "symbols.")
    print("The model is a transformer (DistilBertForSequenceClassification)")

The alphabet contains 22 symbols.
The type of the recurrent cells is RNN


## Load the data

The input data is in the following format :

```
[Number of sequences] [Alphabet size]
[Length of sequence] [List of symbols]
[Length of sequence] [List of symbols]
[Length of sequence] [List of symbols]
...
[Length of sequence] [List of symbols]
```

For example the following data :

```
5 10
6 8 6 5 1 6 7 4 9
12 8 6 9 4 6 8 2 1 0 6 5 9
7 8 9 4 3 0 4 9
4 8 0 4 9
8 8 1 5 2 6 0 5 3 9
```

is composed of 5 sequences and has an alphabet size of 10 (so symbols are between 0 and 9) and the first sequence is composed of 6 symbols (8 6 5 1 6 7 4 9), notice that 8 is the start symbol and 9 is the end symbol.

In [151]:
from pythautomata.base_types.alphabet import Alphabet

file = f"datasets/{TRACK}.{DATASET}.taysir.valid.words"

alphabet = None
sequences = []

#In the competition the empty sequence is defined as [alphabet_size - 2, alphabet size -1]
#For example with the alphabet of size 22 the empty sequence is [20, 21]
empty_sequence_len = 2

with open(file) as f:
    a = f.readline() #Skip first line (number of sequences, alphabet size)
    headline = a.split(' ')
    alphabet_size = int(headline[1].strip())
    alphabet = Alphabet.from_strings([str(x) for x in range(alphabet_size - empty_sequence_len)])
    
    for line in f:
        line = line.strip()
        seq = line.split(' ')
        seq = [int(i) for i in seq[1:]] #Remove first value (length of sequence) and cast to int
        sequences.append(seq)

The variable *sequences* is thus **a list of lists**

In [152]:
print('Number of sequences:', len(sequences))
print('10 first sequences:')
for i in range(10):
    print(sequences[i])

Number of sequences: 10000
10 first sequences:
[16, 13, 13, 13, 4, 6, 3, 9, 0, 0, 13, 11, 8, 2, 8, 13, 4, 8, 15, 14, 15, 17]
[16, 6, 15, 1, 14, 5, 12, 4, 11, 14, 7, 8, 15, 7, 6, 3, 7, 6, 13, 10, 0, 17]
[16, 13, 13, 13, 10, 1, 1, 12, 9, 9, 15, 13, 8, 1, 11, 12, 8, 5, 7, 7, 8, 17]
[16, 13, 13, 13, 2, 12, 0, 15, 8, 4, 4, 13, 5, 11, 13, 2, 4, 8, 0, 12, 5, 17]
[16, 8, 10, 7, 2, 13, 11, 14, 0, 3, 14, 6, 9, 11, 11, 13, 10, 15, 3, 10, 7, 17]
[16, 13, 13, 13, 5, 1, 10, 2, 3, 15, 3, 14, 6, 11, 14, 3, 6, 8, 13, 2, 4, 17]
[16, 9, 7, 3, 2, 6, 0, 0, 5, 8, 0, 8, 10, 5, 13, 12, 0, 14, 14, 2, 11, 17]
[16, 13, 13, 13, 11, 14, 13, 7, 4, 6, 5, 5, 14, 3, 5, 9, 9, 10, 9, 2, 1, 17]
[16, 6, 4, 14, 9, 14, 0, 10, 13, 10, 4, 1, 11, 1, 3, 12, 4, 14, 6, 0, 1, 17]
[16, 13, 13, 13, 8, 7, 1, 0, 1, 0, 2, 10, 10, 15, 13, 8, 12, 4, 3, 6, 5, 17]


## Model extraction

This is where you will extract your simple own model.

In [126]:
from pythautomata.model_comparators.wfa_partition_comparison_strategy import WFAPartitionComparator
from pythautomata.utilities.probability_partitioner import QuantizationProbabilityPartitioner
#from pythautomata.model_exporters.wfa_image_exporter_with_partition_mapper import WFAImageExporterWithPartitionMapper
from pythautomata.base_types.symbol import SymbolStr
from pythautomata.utilities.uniform_length_sequence_generator import UniformLengthSequenceGenerator

from pymodelextractor.learners.observation_tree_learners.bounded_pdfa_quantization_n_ary_tree_learner import BoundedPDFAQuantizationNAryTreeLearner
from pymodelextractor.teachers.pac_batch_probabilistic_teacher import PACBatchProbabilisticTeacher
from pymodelextractor.teachers.pac_probabilistic_teacher import PACProbabilisticTeacher
from pymodelextractor.utils.pickle_data_loader import PickleDataLoader

from utils import predict
from pytorch_language_model import PytorchLanguageModel

name = "track_" + str(TRACK) + "_dataset_" + str(DATASET)

target_model = PytorchLanguageModel(alphabet, model, name)


In [147]:
utils_seq_proba = utils.sequence_probability([0,1,0], model)
utils_seq_proba

0.0013165645068511367

In [105]:
from pythautomata.base_types.alphabet import Alphabet
from pythautomata.base_types.symbol import SymbolStr
from pythautomata.base_types.sequence import Sequence

def get_alphabet_and_validation_sequences(ds):
    file = f"datasets/2.{ds}.taysir.valid.words"
    alphabet = None
    sequences = []

    empty_sequence_len = 2
    with open(file) as f:
        a = f.readline()
        headline = a.split(' ')
        alphabet_size = int(headline[1].strip())
        alphabet = Alphabet.from_strings([str(x) for x in range(alphabet_size - empty_sequence_len)])
        for line in f:
            line = line.strip()
            seq = line.split(' ')[1:]
            seq = [SymbolStr(i) for i in seq[1:]]            
            sequences.append(Sequence(seq))
    return alphabet, sequences

In [106]:
_, pythaut_sequences = get_alphabet_and_validation_sequences(DATASET)

In [107]:
suffixes = [target_model.terminal_symbol]
for symbol in target_model.alphabet.symbols:
    suffixes.append(Sequence((symbol,)))

In [133]:
seq_0 = Sequence([SymbolStr('0')])
seq_010 = Sequence([SymbolStr('0'), SymbolStr('1'), SymbolStr('0')])
seq_eps = Sequence([])

In [131]:
symbols = list(target_model.alphabet.symbols)
symbols.sort()
symbols = [target_model.terminal_symbol] + symbols
target_model.get_last_token_weights(seq_0, symbols)

[0.0281967930495739,
 0.0,
 0.02822423353791237,
 0.028198517858982086,
 0.0281969141215086,
 0.07661447674036026,
 0.028222467750310898,
 0.028376396745443344,
 0.06932707130908966,
 0.028197145089507103,
 0.049477893859148026,
 0.0281967930495739,
 0.04306797683238983,
 0.062456440180540085,
 0.0366104356944561,
 0.07664680480957031,
 0.031008711084723473,
 0.07645978033542633,
 0.07664657384157181,
 0.07245078682899475,
 0.04702847823500633]

In [145]:
target_model.sequence_probability(seq_010)

0.0

In [138]:
target_model.get_last_token_weights(seq_0, symbols)

[0.0281967930495739,
 0.0,
 0.02822423353791237,
 0.028198517858982086,
 0.0281969141215086,
 0.07661447674036026,
 0.028222467750310898,
 0.028376396745443344,
 0.06932707130908966,
 0.028197145089507103,
 0.049477893859148026,
 0.0281967930495739,
 0.04306797683238983,
 0.062456440180540085,
 0.0366104356944561,
 0.07664680480957031,
 0.031008711084723473,
 0.07645978033542633,
 0.07664657384157181,
 0.07245078682899475,
 0.04702847823500633]

In [144]:
utils.sequence_probability([0,1,0], model)

0.0013165645068511367

In [139]:
utils.next_symbols_probas([0,1,0], model)

array([0.03897433, 0.03897433, 0.03900979, 0.1059432 , 0.03897433,
       0.03897433, 0.03948918, 0.04716857, 0.03897458, 0.03897433,
       0.03897433, 0.03897436, 0.03897433, 0.10288801, 0.04191166,
       0.03897433, 0.03897433, 0.03897433, 0.03897433, 0.03897433,
       0.03897433, 0.03897433], dtype=float32)

In [110]:
symbols

[21, 0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 3, 4, 5, 6, 7, 8, 9]

In [111]:
suffixes

[21, 1, 19, 4, 12, 0, 14, 9, 16, 5, 11, 10, 2, 3, 6, 7, 15, 8, 18, 13, 17]

In [112]:
target_model.get_last_token_weights_batch([seq_0], suffixes)

[[0.028196793,
  0.028224234,
  0.043067977,
  0.076646805,
  0.07661448,
  0.0,
  0.028376397,
  0.04702848,
  0.028197145,
  0.031008711,
  0.028196914,
  0.028198518,
  0.06245644,
  0.036610436,
  0.07645978,
  0.076646574,
  0.06932707,
  0.07245079,
  0.028196793,
  0.028222468,
  0.049477894]]

In [113]:
#from last_token_weights_pickle_dataset_generator import LastTokenWeightsPickleDataSetGenerator
#LastTokenWeightsPickleDataSetGenerator().genearte_dataset(target_model, 1000, "./test",10)

In [114]:
print(type(alphabet))

<class 'pythautomata.base_types.alphabet.Alphabet'>


In [141]:
from pythautomata.automata.wheighted_automaton_definition.probabilistic_deterministic_finite_automaton import \
    ProbabilisticDeterministicFiniteAutomaton as PDFA
from pythautomata.automata.wheighted_automaton_definition.weighted_state import WeightedState
from pythautomata.model_comparators.wfa_tolerance_comparison_strategy import WFAToleranceComparator

weight = 1.0/(len(alphabet)+1)
weight = 0
q0 = WeightedState("q0", 1, weight)
for symbol in alphabet.symbols:
    q0.add_transition(symbol, q0, weight)
states = {q0}
comparator = WFAToleranceComparator()
testPDFA = PDFA(alphabet, states, target_model.terminal_symbol, comparator, "Test", check_is_probabilistic = False)


In [142]:
testPDFA.sequence_probability(seq_0)

0.0

In [117]:
epsilon = 0.1
delta = 0.1
max_states = 1000000
max_query_length= 1000000
max_secs = 30
sequence_generator = UniformLengthSequenceGenerator(alphabet, max_seq_length=100, min_seq_length=20)
#dataloader = PickleDataLoader("./test")

partitioner = QuantizationProbabilityPartitioner(10)
comparator = WFAPartitionComparator(partitioner)
teacher1  = PACBatchProbabilisticTeacher(target_model, epsilon = epsilon, delta = delta, max_seq_length = None, comparator = comparator, sequence_generator=sequence_generator, compute_epsilon_star=False)
learner = BoundedPDFAQuantizationNAryTreeLearner(partitioner, max_states, max_query_length, max_secs, generate_partial_hipothesis = False, pre_cache_queries_for_building_hipothesis = False,  check_probabilistic_hipothesis = False)
learning_result = learner.learn(teacher1)     

In [12]:
#teacher2  = PACBatchProbabilisticTeacher(target_model, epsilon = epsilon, delta = delta, max_seq_length = None, comparator = comparator, sequence_generator=sequence_generator, compute_epsilon_star=False, cache_from_dataloader=dataloader)
#learner2 = BoundedPDFAQuantizationNAryTreeLearner(partitioner, max_states, max_query_length, max_secs, generate_partial_hipothesis = False, pre_cache_queries_for_building_hipothesis = False,  check_probabilistic_hipothesis = False)
#learning_result = learner2.learn(teacher2)     

In [13]:
learning_result.model

## Save and submit 
This is the creation of the model needed for the submission to the competition: you just have to run this cell. It will create in your current directory an **archive** that you can then submit on the competition website.

**You should NOT modify this part, just run it**

In [14]:
from fast_pdfa_wrapper import MlflowFastPDFA
from submit_tools_fix import save_function
from fast_pdfa_converter import FastProbabilisticDeterministicFiniteAutomatonConverter as FastPDFAConverter

fast_pdfa = FastPDFAConverter().to_fast_pdfa(learning_result.model)
mlflow_fast_pdfa = MlflowFastPDFA(fast_pdfa)
save_function(mlflow_fast_pdfa, len(learning_result.model.alphabet), target_model.name)

Submission created at predicted_models/track_2_dataset_10.zip.


/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [143]:
from fast_pdfa_wrapper import MlflowFastPDFA
from submit_tools_fix import save_function
from fast_pdfa_converter import FastProbabilisticDeterministicFiniteAutomatonConverter as FastPDFAConverter

fast_pdfa = FastPDFAConverter().to_fast_pdfa(testPDFA)
mlflow_fast_pdfa = MlflowFastPDFA(fast_pdfa)
save_function(mlflow_fast_pdfa, len(testPDFA.alphabet), "TEST_PDFA_0")

Submission created at predicted_models/TEST_PDFA_0.zip.


In [97]:
save_function(mlflow_fast_pdfa, len(testPDFA.alphabet), "TEST_PDFA")

Submission created at predicted_models/TEST_PDFA.zip.


In [98]:
#zip_path = f"predicted_models/{target_model.name}.zip"
zip_path = f"predicted_models/TEST_PDFA.zip"
from load_func import load_function
print(sequences[0:10])
load_function(zip_path, sequences[0:10])

[[20, 19, 8, 18, 7, 19, 11, 3, 18, 11, 2, 16, 7, 9, 8, 6, 15, 7, 9, 16, 2, 8, 15, 6, 13, 14, 19, 8, 8, 7, 15, 3, 9, 6, 10, 6, 17, 15, 21], [20, 7, 13, 12, 6, 8, 13, 9, 8, 3, 16, 7, 12, 13, 14, 7, 7, 13, 16, 15, 9, 13, 19, 19, 7, 2, 4, 2, 15, 15, 12, 8, 7, 11, 7, 17, 15, 21], [20, 11, 13, 19, 18, 11, 9, 13, 11, 4, 7, 10, 2, 14, 9, 6, 2, 2, 19, 1, 12, 3, 11, 10, 6, 3, 2, 3, 2, 5, 15, 11, 8, 7, 18, 14, 17, 15, 21], [20, 3, 9, 19, 10, 19, 6, 5, 2, 11, 1, 12, 7, 17, 4, 7, 15, 17, 14, 2, 18, 12, 18, 6, 13, 4, 2, 10, 15, 15, 3, 8, 7, 1, 14, 21], [20, 2, 11, 2, 19, 2, 13, 9, 9, 12, 3, 4, 12, 8, 7, 13, 17, 13, 16, 10, 11, 1, 6, 6, 19, 3, 2, 15, 5, 8, 15, 19, 3, 11, 11, 6, 7, 6, 13, 2, 3, 3, 4, 15, 12, 6, 7, 11, 14, 17, 21], [20, 18, 16, 6, 5, 0, 19, 2, 13, 9, 11, 3, 8, 14, 12, 9, 14, 13, 14, 2, 2, 15, 3, 18, 10, 13, 13, 8, 13, 3, 12, 15, 18, 8, 7, 11, 7, 19, 15, 21], [20, 1, 5, 14, 5, 7, 19, 12, 3, 5, 14, 11, 3, 14, 2, 2, 7, 1, 10, 2, 16, 19, 7, 14, 11, 12, 2, 14, 3, 12, 2, 12, 7, 14, 4, 2, 3, 